In [0]:
# ============================================
# Power BI View #1: KPI Summary
# ============================================

spark.sql("""
CREATE OR REPLACE VIEW fraud_detection.raw.powerbi_fraud_summary AS
SELECT 
    COUNT(*) as total_fraud_alerts,
    ROUND(SUM(amount), 2) as total_fraud_amount,
    COUNT(DISTINCT card_id) as unique_cards_flagged,
    ROUND(AVG(speed_kmh), 0) as avg_impossible_speed,
    ROUND(MAX(speed_kmh), 0) as max_speed_detected,
    SUM(CASE WHEN severity = 'CRITICAL' THEN 1 ELSE 0 END) as critical_alerts,
    SUM(CASE WHEN severity = 'HIGH' THEN 1 ELSE 0 END) as high_alerts,
    MAX(alert_timestamp) as last_alert_time
FROM fraud_detection.raw.gold_fraud_alerts
WHERE alert_timestamp >= CURRENT_TIMESTAMP() - INTERVAL 24 HOUR
""")

print("✅ View created: powerbi_fraud_summary")
print("   Use for: KPI cards (total alerts, amount, cards flagged)")


✅ View created: powerbi_fraud_summary
   Use for: KPI cards (total alerts, amount, cards flagged)


In [0]:
# ============================================
# Power BI View #2: Hourly Fraud Trends
# ============================================

spark.sql("""
CREATE OR REPLACE VIEW fraud_detection.raw.powerbi_fraud_trends AS
SELECT 
    DATE_TRUNC('hour', alert_timestamp) as hour,
    COUNT(*) as fraud_count,
    ROUND(SUM(amount), 2) as total_amount,
    ROUND(AVG(speed_kmh), 0) as avg_speed,
    COUNT(DISTINCT card_id) as unique_cards,
    severity
FROM fraud_detection.raw.gold_fraud_alerts
WHERE alert_timestamp >= CURRENT_TIMESTAMP() - INTERVAL 7 DAY
GROUP BY DATE_TRUNC('hour', alert_timestamp), severity
""")

print("✅ View created: powerbi_fraud_trends")
print("   Use for: Line chart (fraud over time)")


✅ View created: powerbi_fraud_trends
   Use for: Line chart (fraud over time)


In [0]:
# ============================================
# Power BI View #4: Top Fraud Cards
# ============================================

spark.sql("""
CREATE OR REPLACE VIEW fraud_detection.raw.powerbi_top_fraud_cards AS
SELECT 
    card_id,
    COUNT(*) as fraud_count,
    ROUND(SUM(amount), 2) as total_amount,
    ROUND(AVG(speed_kmh), 0) as avg_speed,
    MAX(severity) as max_severity,
    MAX(alert_timestamp) as last_alert
FROM fraud_detection.raw.gold_fraud_alerts
WHERE alert_timestamp >= CURRENT_TIMESTAMP() - INTERVAL 7 DAY
GROUP BY card_id
ORDER BY fraud_count DESC
LIMIT 20
""")

print("✅ View created: powerbi_top_fraud_cards")
print("   Use for: Table visual (top fraud cards)")


✅ View created: powerbi_top_fraud_cards
   Use for: Table visual (top fraud cards)


In [0]:
# ============================================
# Power BI View #5: System Performance
# ============================================

spark.sql("""
CREATE OR REPLACE VIEW fraud_detection.raw.powerbi_system_performance AS
SELECT 
    batch_timestamp,
    transactions_processed,
    fraud_detected,
    ROUND(processing_time_sec, 2) as processing_sec,
    ROUND(avg_latency_ms, 0) as latency_ms,
    ROUND((fraud_detected * 100.0 / NULLIF(transactions_processed, 0)), 2) as fraud_rate_pct
FROM fraud_detection.raw.gold_system_performance
WHERE batch_timestamp >= CURRENT_TIMESTAMP() - INTERVAL 24 HOUR
ORDER BY batch_timestamp DESC
""")

print("✅ View created: powerbi_system_performance")
print("   Use for: Line chart (latency over time)")


✅ View created: powerbi_system_performance
   Use for: Line chart (latency over time)


In [0]:
# ============================================
# Test All Power BI Views
# ============================================

print("\n📊 TESTING ALL POWER BI VIEWS")
print("=" * 70)

views = [
    "powerbi_fraud_summary",
    "powerbi_fraud_trends",
    "powerbi_fraud_geography",
    "powerbi_top_fraud_cards",
    "powerbi_system_performance"
]

for view in views:
    count = spark.table(f"fraud_detection.raw.{view}").count()
    print(f"✅ {view}: {count} rows")

print("\n" + "=" * 70)
print("✅ ALL VIEWS READY FOR POWER BI!")
print("\n📝 Power BI Connection Details:")
print("   Server: <your-workspace>.azuredatabricks.net")
print("   HTTP Path: /sql/1.0/warehouses/<warehouse-id>")
print("   Catalog: fraud_detection")
print("   Schema: raw")
print("\n💡 In Power BI, connect to these 5 views")



📊 TESTING ALL POWER BI VIEWS
✅ powerbi_fraud_summary: 1 rows
✅ powerbi_fraud_trends: 2 rows


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4990283572869452>, line 17
      8 views = [
      9     "powerbi_fraud_summary",
     10     "powerbi_fraud_trends",
   (...)
     13     "powerbi_system_performance"
     14 ]
     16 for view in views:
---> 17     count = spark.table(f"fraud_detection.raw.{view}").count()
     18     print(f"✅ {view}: {count} rows")
     20 print("\n" + "=" * 70)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1917, in SparkSession.table(self, tableName)
   1911 if